In [1]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai gradio numpy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.4 MB/s eta 

In [2]:
import os
from getpass import getpass
import time
import sqlite3
import numpy as np
import gradio as gr

from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

In [3]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter Mistral API Key: ")

llm = ChatMistralAI(model="mistral-large-latest", temperature=0)
embeddings = MistralAIEmbeddings()

Enter Mistral API Key: ··········


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [14]:
DB_PATH = "/content/rag_memory.db"

def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS chat_history (
            session_id TEXT,
            user_message TEXT,
            bot_message TEXT,
            citations TEXT,
            confidence REAL,
            response_time REAL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    conn.close()

init_db()

In [15]:
current_vectorstore = None
current_pdf_name = None


In [16]:
def process_pdf(file):
    global current_vectorstore, current_pdf_name

    if file is None:
        return "❌ Please upload a PDF file."

    loader = PyPDFLoader(file.name)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=100
    )
    chunks = splitter.split_documents(docs)

    current_vectorstore = FAISS.from_documents(chunks, embeddings)
    current_pdf_name = os.path.basename(file.name)

    return f"✅ PDF processed successfully: **{current_pdf_name}**\nChunks created: {len(chunks)}"

In [17]:
def confidence_score(scores):
    if not scores:
        return 0.0
    avg = sum(scores) / len(scores)
    return round(max(0.0, min(1.0, 1 - avg)), 2)

In [18]:
def save_chat(session_id, user, bot, citations, confidence, response_time):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO chat_history
        (session_id, user_message, bot_message, citations, confidence, response_time)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (session_id, user, bot, citations, confidence, response_time))
    conn.commit()
    conn.close()

In [19]:
rag_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Answer the question ONLY using the context below.
    Cite sources as [page X].

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
)

In [20]:
def chat_fn(message, session_id):
    global current_vectorstore

    if current_vectorstore is None:
        return "❌ Please upload and process a PDF first."

    start = time.time()

    docs_scores = current_vectorstore.similarity_search_with_score(message, k=5)
    docs = [d for d, _ in docs_scores]
    scores = [s for _, s in docs_scores]

    context = "\n\n".join(
        [f"[page {d.metadata.get('page', '?')}] {d.page_content}" for d in docs]
    )

    response = llm.invoke(
        rag_prompt.format(context=context, question=message)
    )

    conf = confidence_score(scores)
    citations = ", ".join(
        [f"page {d.metadata.get('page', '?')}" for d in docs[:3]]
    )

    elapsed = round(time.time() - start, 2)

    save_chat(session_id, message, response.content, citations, conf, elapsed)

    footer = f"""
---
**📄 Document:** {current_pdf_name}
**🔍 Confidence:** {conf}
**📚 Sources:** {citations}
**⏱ Response Time:** {elapsed}s
"""

    return response.content + footer

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 📘 Smart RAG Assistant (Single PDF)")

    with gr.Row():
        pdf_file = gr.File(label="Upload PDF", file_types=[".pdf"])
        process_btn = gr.Button("Process Document")

    status = gr.Markdown()

    process_btn.click(
        fn=process_pdf,
        inputs=pdf_file,
        outputs=status
    )

    gr.Markdown("### 💬 Ask questions from the uploaded PDF")

    question = gr.Textbox(label="Your question")
    session_id = gr.Textbox(value="session_1", label="Session ID")
    answer = gr.Markdown()

    submit = gr.Button("Ask")

    submit.click(
        fn=chat_fn,
        inputs=[question, session_id],
        outputs=answer
    )

demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://cd1cec6f7b05a7bea7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/httptools_impl.py", line 409, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/error